In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np

In [ ]:
# Path where the model and tokenizer were saved
model_path = '/content/drive/MyDrive/NLU_Files/final_roberta_optimal_lr'

# Load the model
model = RobertaForSequenceClassification.from_pretrained(model_path)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_path)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prediction function takes an evidence and a claim, and outputs the prediction
def predict(sentence1, sentence2):
    inputs = tokenizer(sentence1, sentence2, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1).item()
    return prediction

# This section demonstrates an example sentence and a claim

In [ ]:
sentence1 = "We should further exploit nuclear power"
sentence2 = "In 1990 the United States Congress requested the National Cancer Institute to conduct a study of cancer mortality rates around nuclear plants and other facilities covering 1950 to 1984 focusing on the change after operation started of the respective facilities. They concluded in no link"
output = predict(sentence1, sentence2)
print("Prediction:", output)

Prediction: 1


# This section can load in a csv and ouptut a predictions.csv file with the output

In [ ]:
def predict_from_csv(csv_link, model, tokenizer, device):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_link)

    # List to store predictions
    predictions = []

    # Iterate over each row in the DataFrame and make predictions
    for index, row in df.iterrows():
        claim = row['Claim']
        evidence = row['Evidence']
        prediction = predict(claim, evidence)
        predictions.append(prediction)

    # Create a new DataFrame with the predictions
    predictions_df = pd.DataFrame(predictions, columns=['prediction'])

    predictions_df.to_csv('predictions.csv', index=False)

link = "/content/drive/MyDrive/NLU_Files/dev.csv"
predict_from_csv(link, model, tokenizer, device)
